In [1]:
# Dependendcies
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
from splinter import Browser

In [2]:
executable_path = {'executable_path' : '/Users/krist/Downloads/chromedriver_win32/chromedriver'}
browser = Browser("chrome", **executable_path, headless = False)

# NASA Mars News

In [3]:
# scrape the site (maybe use splinter?)
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [4]:
# Create BautifulSoup object; parse with 'html.parser'
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# First news title and paragraph
news_title = soup.find_all('div', class_="content_title")[1].text
news_p = soup.find('div', class_='article_teaser_body').text

print(news_title)
print(news_p)

NASA's Perseverance Drives on Mars' Terrain for First Time
The first trek of the agency’s largest, most advanced rover yet on the Red Planet marks a major milestone before science operations get under way.


# JPL Mars Space Images - Featured Image

In [6]:
# scrape the site
url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url)

In [7]:
# Get the image html
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
image=soup.find('a', class_='showimg')['href']
image

'image/featured/mars1.jpg'

In [8]:
featured_image_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/'+ image
featured_image_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg'

# Mars Facts

In [9]:
# Scrape Mars facts tables
url = 'https://space-facts.com/mars/'
tables = pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [10]:
# find the table with mars only information
df = tables[0]
df.head()

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [11]:
mars_facts = df.to_html()

# Mars Hemispheres 

In [73]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [43]:
# Hemisphere Name
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

h_names = []

results = soup.find_all('div', class_='item')

for result in results:
    hemisphere = result.find('h3')
    h_names.append(hemisphere.text)
    
h_names

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [58]:
# Get thumbnail link
html=browser.html
soup=BeautifulSoup(html, 'html.parser')

t_link = []

for result in results:
    t_result = result.find('a')['href']
    t_url = 'https://astrogeology.usgs.gov/' + t_result
    t_link.append(t_url)

t_link 

['https://astrogeology.usgs.gov//search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov//search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov//search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov//search/map/Mars/Viking/valles_marineris_enhanced']

In [62]:
# Parse through thumnails for full-size image link
hemisphere_img_urls = []

for link in t_link:
    browser.visit(link)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    images = soup.find('img', class_='wide-image')['src']
    img_link = 'https://astrogeology.usgs.gov/' + images
    
    hemisphere_img_urls.append(img_link)

hemisphere_img_urls

['https://astrogeology.usgs.gov//cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov//cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov//cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov//cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg']

In [72]:
hemi_zip = zip(h_names, hemisphere_img_urls)
hemisphere_dict = []

for title, img in hemi_zip:
    mars_dict = {}
    mars_dict['title'] = title
    mars_dict['img_url'] = img
    hemisphere_dict.append(mars_dict)
hemisphere_dict

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]